In [2]:
import imaplib
import email

# create an IMAP client instance
mail = imaplib.IMAP4_SSL('imap.gmail.com')

# specify the keyword to search for
email_address = input('what is your email address?: ')
password = input('what is your App password? (This is a special password under security under settings): ')
keyword_to_search = input('what keyword do you want to search?:')

# log in to the Gmail account
mail.login(email_address, password)

# select the inbox folder
mail.select('inbox')

# get the latest 150 email messages from the inbox
typ, data = mail.search(None, 'ALL')
latest_email_ids = data[0].split()[-150:]

print(f'Searching for keyword "{keyword_to_search}" in the body of the latest 150 email messages...')

count_keyword = 0
sender_list = [] # create an empty list to store email addresses of senders

# iterate over the latest 150 email messages
for email_id in latest_email_ids:
    typ, msg_data = mail.fetch(email_id, '(RFC822)')
    for response_part in msg_data:
        if isinstance(response_part, tuple):
            # parse the email message
            msg = email.message_from_bytes(response_part[1])
            
            # get the sender of the email
            sender = msg['From']
            
            # extract the body of the email as text
            if msg.is_multipart():
                for part in msg.walk():
                    if part.get_content_type() == 'text/plain':
                        try:
                            body = part.get_payload(decode=True).decode('utf-8')
                        except UnicodeDecodeError:
                            body = part.get_payload(decode=True).decode('iso-8859-1')
                        count_keyword += body.count(keyword_to_search)
                        
                        # append the sender's email address to the list if the keyword is found
                        if keyword_to_search.lower() in body.lower() and sender not in sender_list:
                            sender_list.append(sender)
                            
                    elif part.get_content_maintype() == 'text':
                        try:
                            body = part.get_payload(decode=True).decode('utf-8')
                        except UnicodeDecodeError:
                            body = part.get_payload(decode=True).decode('iso-8859-1')
                        count_keyword += body.count(keyword_to_search)
                        
                        # append the sender's email address to the list if the keyword is found
                        if keyword_to_search.lower() in body.lower() and sender not in sender_list:
                            sender_list.append(sender)
            else:
                try:
                    body = msg.get_payload(decode=True).decode('utf-8')
                except UnicodeDecodeError:
                    body = msg.get_payload(decode=True).decode('iso-8859-1')
                count_keyword += body.count(keyword_to_search)
                
                # append the sender's email address to the list if the keyword is found
                if keyword_to_search.lower() in body.lower() and sender not in sender_list:
                    sender_list.append(sender)

# print the total number of occurrences of the keyword
print(f'The keyword "{keyword_to_search}" appears {count_keyword} time(s) in the body of the latest 150 email messages.')
print('Senders of emails containing the keyword:')
print(sender_list)

what is your email address?: 5291ccc@gmail.com
what is your App password? (This is a special password under security under settings): oofsbpjgwwyhipto
what keyword do you want to search?:review
Searching for keyword "review" in the body of the latest 150 email messages...
The keyword "review" appears 231 time(s) in the body of the latest 150 email messages.
Senders of emails containing the keyword:
['"Your Indeed Job Feed" <alert@indeed.com>', 'Jungle Scout <hello@junglescoutmail.com>', 'International TEFL Academy <jeff@internationalteflacademy.com>', '"Amazon.com" <auto-confirm@amazon.com>', '"Amazon.com" <shipment-tracking@amazon.com>', '"Sony Rewards" <donotreply@member.sonyrewards.com>', 'Headway <noreply@findheadway.com>', 'Robinhood <noreply@robinhood.com>', '"Amazon.com" <donotreply@amazon.com>', 'Amazon Marketplace <marketplace-messages@amazon.com>', '=?utf-8?Q?Vincent=20Delaunay=20&=20The=20Anglo=2DTeam?= <info@angloville.com>', '"PayPal" <PayPal@emails.paypal.com>', '"=?US-AS